In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [3]:
bookmark_url = 'https://www.yelp.com/user_details_bookmarks?userid='
review_url = 'https://www.yelp.com/user_details_reviews_self?userid='
friends_url = 'https://www.yelp.com/user_details_friends?userid='
self_id = 'ShHBKjuJbQAVBLs7DgA95A'

In [6]:
def first_page_friends(user_id):
    """function to get the User IDs of the first page of friends"""

    f_page = requests.get(friends_url + user_id)
    f_soup = BeautifulSoup(f_page.content, 'html.parser')
    f_source = f_soup.find_all('a', class_='user-display-name js-analytics-click')
    
    friend_list = []
    for friend in range(0, len(f_source)):
        friend_id = f_source[friend]['href'].split('=')[1]
        friend_list.append(friend_id)
    
    return friend_list

In [19]:
def get_bookmarks(user_id):
    """get all bookmarks for user_id"""
    
    # get the max page number of bookmarks
    bookmarks = requests.get(bookmark_url+user_id)
    bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
    pages = bm_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
    max_page = pages[0].get_text().split('of ')[1].split('\n')[0]
    
    user_bookmarks = []
    # loop through all the pages to get bookmarks
    for page_num in range(0, 50*int(max_page), 50):
        bookmarks = requests.get(bookmark_url + user_id + '&start=' + str(page_num))
        bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
        bm_name = bm_soup.find_all('a', class_='biz-name js-analytics-click')
        
        for mark in range(0, len(bm_name)):
            user_bookmarks.append(bm_name[mark]['href'])
       
    return list(set(user_bookmarks))

In [20]:
self_bookmarks = get_bookmarks(self_id)
self_bookmarks

['/biz/ten-ichi-san-francisco',
 '/biz/gaku-yakitori-san-jose',
 '/biz/kamei-restaurant-supply-san-francisco',
 '/biz/koi-palace-daly-city',
 '/biz/sues-hair-studio-san-francisco',
 '/biz/mingalaba-burlingame',
 '/biz/rainbow-grocery-san-francisco-2',
 '/biz/theorita-san-francisco',
 '/biz/che-fico-san-francisco-3',
 '/biz/boulevard-san-francisco',
 '/biz/birdsong-san-francisco',
 '/biz/hodala-san-francisco',
 '/biz/la-costanera-montara',
 '/biz/leichi-santa-clara-2',
 '/biz/pasta-armellino-saratoga-3',
 '/biz/han-il-kwan-san-francisco',
 '/biz/orchard-city-kitchen-campbell',
 '/biz/xiao-long-bao-kitchen-south-san-francisco',
 '/biz/dumpling-empire-south-san-francisco',
 '/biz/bakesale-betty-oakland',
 '/biz/milq-tea-bar-and-lounge-san-jose-4',
 '/biz/spice-kit-san-francisco',
 '/biz/dancing-bull-san-francisco',
 '/biz/alta-san-francisco',
 '/biz/asha-tea-house-san-francisco',
 '/biz/asha-tea-house-berkeley',
 '/biz/green-healthy-massage-santa-clara',
 '/biz/red-hot-wok-cupertino',
 '/

In [55]:
%store self_bookmarks

Stored 'self_bookmarks' (list)


In [21]:
len(self_bookmarks)

248

In [8]:
source_friends = first_page_friends(self_id)

In [9]:
len(source_friends)

48

In [32]:
import re
reviews = requests.get(review_url+self_id)
re_soup = BeautifulSoup(reviews.content, 'html.parser')
pages = re_soup.find_all('div', class_=re.compile('i-stars i-stars--regular-*'))
pages

[<div class="i-stars i-stars--regular-5 rating-large" title="5.0 star rating">
 <img alt="5.0 star rating" class="offscreen" height="303" src="https://s3-media2.fl.yelpcdn.com/assets/srv0/yelp_design_web/9b34e39ccbeb/assets/img/stars/stars.png" width="84"/>
 </div>,
 <div class="i-stars i-stars--regular-5 rating-large" title="5.0 star rating">
 <img alt="5.0 star rating" class="offscreen" height="303" src="https://s3-media2.fl.yelpcdn.com/assets/srv0/yelp_design_web/9b34e39ccbeb/assets/img/stars/stars.png" width="84"/>
 </div>,
 <div class="i-stars i-stars--regular-3 rating-large" title="3.0 star rating">
 <img alt="3.0 star rating" class="offscreen" height="303" src="https://s3-media2.fl.yelpcdn.com/assets/srv0/yelp_design_web/9b34e39ccbeb/assets/img/stars/stars.png" width="84"/>
 </div>,
 <div class="i-stars i-stars--regular-3 rating-large" title="3.0 star rating">
 <img alt="3.0 star rating" class="offscreen" height="303" src="https://s3-media2.fl.yelpcdn.com/assets/srv0/yelp_design

In [60]:
# create dictionary of users and their bookmarks

all_bookmarks = {i:[] for i in source_friends}

for i in source_friends:
    all_bookmarks[i] = get_bookmarks(i)

In [61]:
%store all_bookmarks

Stored 'all_bookmarks' (dict)


In [102]:
bm_vocab = [item for sl in list(all_bookmarks.values()) for item in sl]

In [103]:
bm_set = list(set(bm_vocab))
len(bm_set)

5260

In [104]:
all_usr_vector = []

for k,v in all_bookmarks.items():
    usr_vector = []
    for bm in bm_set:
        if bm in v:
            usr_vector.extend([1])
        else:
            usr_vector.extend([0])
    all_usr_vector.append(usr_vector)

In [136]:
len(all_usr_vector)

48

In [140]:
bm_df = pd.DataFrame(all_usr_vector, columns=bm_set, index=all_bookmarks.keys())
bm_df.head()

,Cafe Birdie,Spicy King,Bar Pa Tea,Bao Bar,Magowan’s Infinite Mirror Maze,SOHO,Gao Viet Kitchen & Bar,Klunkerkranich,Bombay Tandoori & Banquet,The Riddler,...,Taiwan Cafe,Giddy,Shin-Sen-Gumi Hakata Ramen-Gardena,Gibson,Sophie’s Crepes,The Sofa Company,Poke Etc,Foreign Cinema,Perilla,Adana
7IV3JXTXufasWNSkDEdrcQ,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5RifcJP_Lf-MzojTHybBNw,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
NmdKCZlUVJGnqDxhXl2ofw,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
QjLiYeQLeMIqYu-ncmkUXg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tdowJfZymyZNJymLmBfZeA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
my_bm_comp = []
for i in bm_df.columns:
    if i in self_bookmarks:
        my_bm_comp.append(1)
    else:
        my_bm_comp.append(0)

In [146]:
mbm = pd.Series(my_bm_comp).to_frame(self_id).T
mbm.columns = bm_df.columns
mbm

,Cafe Birdie,Spicy King,Bar Pa Tea,Bao Bar,Magowan’s Infinite Mirror Maze,SOHO,Gao Viet Kitchen & Bar,Klunkerkranich,Bombay Tandoori & Banquet,The Riddler,...,Taiwan Cafe,Giddy,Shin-Sen-Gumi Hakata Ramen-Gardena,Gibson,Sophie’s Crepes,The Sofa Company,Poke Etc,Foreign Cinema,Perilla,Adana
ShHBKjuJbQAVBLs7DgA95A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [148]:
bookmark_df = pd.concat([bm_df, mbm])
bookmark_df

,Cafe Birdie,Spicy King,Bar Pa Tea,Bao Bar,Magowan’s Infinite Mirror Maze,SOHO,Gao Viet Kitchen & Bar,Klunkerkranich,Bombay Tandoori & Banquet,The Riddler,...,Taiwan Cafe,Giddy,Shin-Sen-Gumi Hakata Ramen-Gardena,Gibson,Sophie’s Crepes,The Sofa Company,Poke Etc,Foreign Cinema,Perilla,Adana
7IV3JXTXufasWNSkDEdrcQ,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5RifcJP_Lf-MzojTHybBNw,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
NmdKCZlUVJGnqDxhXl2ofw,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
QjLiYeQLeMIqYu-ncmkUXg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tdowJfZymyZNJymLmBfZeA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LqLXDCFQfiWzKlKyOhSmZg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jcWQTnEKfWNVeLDTmwb6cQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yx3eatgbMTnzzzAcnBjxCQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
YgTleYno1TQjVr3aCBPgvw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
ayUH6WAwRDpUrlGVIvctFQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
l1 = bookmark_df.iloc[-1,:][bookmark_df.iloc[-1,:]==1].index.values
l2 = self_bookmarks

In [159]:
a = set(l1)
b = set(l2)
b.difference(a)

{'Aarna Threading Salon',
 'Alta',
 'Amazebowls',
 'Apothèke',
 'Asha Tea House',
 'Banana Leaf Restaurant',
 'Banzai Bowls',
 'Baonecci Ristorante',
 'Bar Bocce',
 'Barebottle Brewing Company',
 'Ben Thai Cafe',
 'Bencotto Italian Kitchen',
 'Bing Haus',
 'Bistro Don Giovanni',
 'Bobo’s',
 'Bowl’D Acai',
 'Brick & Bell Cafe - La Jolla Shores',
 'Butter Love Bakeshop',
 'CamboWan',
 'Chile Pies Baking Co.',
 'Chotto',
 'City Dance Studios',
 'Cupcake Royale',
 'Darda Seafood Restaurant',
 'Del Popolo',
 'Delarosa',
 'Dumpling Kitchen',
 'Eastern Bakery',
 'El Milagro',
 'Farm : Table',
 'Fillmore Bakeshop',
 'Fino Ristorante & Bar',
 'Five Happiness',
 'Flora Grubb Gardens',
 'Flowerboy Project',
 'Freddie’s Sandwiches',
 'Gaku Yakitori',
 'Go Fish Poke Bar',
 'Gochi Japanese Fusion Tapas',
 'Grand Lake Kitchen',
 'Great China',
 'Green Healthy Massage',
 'Greenbox',
 'Heritage House Resort & Spa',
 'House Of Pancakes',
 'Huge Tree Pastry',
 'Iza Ramen Lower Haight',
 'Joe’s Pizza',
 '

In [160]:
from sklearn.metrics import jaccard_similarity_score
from scipy.stats import pearsonr

In [161]:
sim_score = {}
for i in range(len(bookmark_df)):
    ss = pearsonr(bookmark_df.iloc[-1,:], bookmark_df.iloc[i,:])
    sim_score.update({i: ss[0]})

/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [162]:
sf = pd.Series(sim_score).to_frame('similarity')

In [164]:
sf.sort_values('similarity', ascending=False)

,similarity
48,1.000000
25,0.097263
16,0.096543
11,0.084261
27,0.071746
2,0.070664
8,0.059837
20,0.054375
18,0.053883
43,0.049128


In [167]:
bookmark_df.iloc[25,:][bookmark_df.iloc[25,:]==1]

Ijji Sushi                                    1
Ajito                                         1
Dragon Beaux                                  1
Red Hot Wok                                   1
The Crack Shack                               1
Delage                                        1
Suppenküche                                   1
Sushi Zone                                    1
Siam Fine Thai Cuisine                        1
Rooster & Rice                                1
Aloha Fresh                                   1
Capital One Café                              1
KUSAKABE                                      1
Ape Do Good Screen Printing                   1
Hot Sauce and Panko                           1
The Grove - Design District                   1
Whitechapel SF                                1
New India Bazar                               1
Chef Chu’s                                    1
Nick’s Crispy Tacos                           1
AVERY                                   

In [168]:
all_recs = bookmark_df.iloc[[25, 16, 11, 48],:][bookmark_df.iloc[[25, 16, 11, 48],:]==1].fillna(0).T
all_recs

,-OKmukwdCrHq6bkF2_gSwQ,Zw-0Lo01W0QdkHT60DYg4g,UaKdT4twgZ4DguHJhT6vPw,ShHBKjuJbQAVBLs7DgA95A
Cafe Birdie,0.0,0.0,0.0,0.0
Spicy King,0.0,0.0,1.0,0.0
Bar Pa Tea,0.0,0.0,0.0,0.0
Bao Bar,0.0,0.0,0.0,0.0
Magowan’s Infinite Mirror Maze,0.0,0.0,0.0,0.0
SOHO,0.0,0.0,0.0,0.0
Gao Viet Kitchen & Bar,0.0,0.0,0.0,0.0
Klunkerkranich,0.0,0.0,0.0,0.0
Bombay Tandoori & Banquet,0.0,0.0,0.0,0.0
The Riddler,0.0,0.0,0.0,0.0


In [170]:
temp = all_recs[all_recs[self_id]==0].copy()
str_recs = temp.iloc[:,:-1].copy()
str_recs

,-OKmukwdCrHq6bkF2_gSwQ,Zw-0Lo01W0QdkHT60DYg4g,UaKdT4twgZ4DguHJhT6vPw
Cafe Birdie,0.0,0.0,0.0
Spicy King,0.0,0.0,1.0
Bar Pa Tea,0.0,0.0,0.0
Bao Bar,0.0,0.0,0.0
Magowan’s Infinite Mirror Maze,0.0,0.0,0.0
SOHO,0.0,0.0,0.0
Gao Viet Kitchen & Bar,0.0,0.0,0.0
Klunkerkranich,0.0,0.0,0.0
Bombay Tandoori & Banquet,0.0,0.0,0.0
The Riddler,0.0,0.0,0.0


In [171]:
str_recs[str_recs.sum(axis=1)>1]

,-OKmukwdCrHq6bkF2_gSwQ,Zw-0Lo01W0QdkHT60DYg4g,UaKdT4twgZ4DguHJhT6vPw
Bask,0.0,1.0,1.0
BBH Noodle,1.0,1.0,0.0
Polly Ann Ice Cream,0.0,1.0,1.0
SUGARFISH by sushi nozawa,0.0,1.0,1.0
Dan Sung Sa,0.0,1.0,1.0
Muracci’s Japanese Curry & Grill,1.0,0.0,1.0
Hamano Sushi,1.0,1.0,0.0
Chubby Noodle,0.0,1.0,1.0
HRD,1.0,1.0,0.0
Izakaya Roku,1.0,1.0,0.0


In [77]:
def get_reviews(user_id):
    """get all reviews for user_id"""
    
    # get the max page number of bookmarks
    reviews = requests.get(review_url+user_id)
    re_soup = BeautifulSoup(reviews.content, 'html.parser')
    pages = re_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
    max_page = pages[0].get_text().split('of ')[1].split('\n')[0]
    
    user_reviews = {}
    # loop through all the pages to get bookmarks
    for page_num in range(0, 10*int(max_page), 10):
        rev = requests.get(review_url + user_id + '&rec_pagestart=' + str(page_num))
        rev_soup = BeautifulSoup(rev.content, 'html.parser')
        rating = rev_soup.find_all('div', class_=re.compile('i-stars i-stars--regular-*'))
        biz_name = rev_soup.find_all('a', class_='biz-name js-analytics-click')
        
        for mark in range(0, len(biz_name)):
            user_reviews[biz_name[mark]['href']] = int(rating[mark]['title'][0])
       
    return user_reviews

In [78]:
my_reviews = get_reviews(self_id)
my_reviews

{'/biz/ace-wasabi-rock-n-roll-sushi-san-francisco': 5,
 '/biz/adventure-cat-sailing-charters-san-francisco-2': 5,
 '/biz/aji-peruvian-cuisine-long-beach-3': 4,
 '/biz/appethaizing-portland': 4,
 '/biz/baohaus-new-york-2': 4,
 '/biz/be-fresh-la-jolla': 5,
 '/biz/beni-tora-los-angeles-3': 4,
 '/biz/cafe-21-san-diego-2': 5,
 '/biz/caffe-centro-san-francisco': 3,
 '/biz/champa-garden-san-francisco-2': 3,
 '/biz/cherry-blossom-bakery-san-francisco-2': 5,
 '/biz/china-stix-restaurant-santa-clara': 5,
 '/biz/chouchou-new-york-2': 3,
 '/biz/cupertino-iphone-repair-cupertino-5': 2,
 '/biz/delicious-food-corner-monterey-park': 2,
 '/biz/elmers-restaurant-portland-3': 3,
 '/biz/em-lash-and-brow-san-jose': 5,
 '/biz/farallon-san-francisco': 3,
 '/biz/fuji-sukiyaki-san-mateo': 3,
 '/biz/furlong-vision-correction-medical-center-san-jose': 5,
 '/biz/genwa-korean-bbq-beverly-hills': 5,
 '/biz/gogigo-korean-bbq-cupertino': 3,
 '/biz/hinata-san-francisco': 5,
 '/biz/ifixers-iphone-repair-san-jose-18': 5

In [80]:
%store my_reviews

Stored 'my_reviews' (dict)


In [79]:
len(my_reviews.keys())

67

In [81]:
# create dictionary of users and their reviews
all_reviews = {i:[] for i in source_friends}

for i in source_friends:
    all_reviews[i] = get_reviews(i)

In [82]:
%store all_reviews

Stored 'all_reviews' (dict)


In [86]:
all_reviews

{'-OKmukwdCrHq6bkF2_gSwQ': {'/biz/la-victoria-taqueria-san-jose-2': 2,
  '/biz/mancinis-sleepworld-colma': 5,
  '/biz/tavor-plumbing-san-bruno': 1},
 '5RifcJP_Lf-MzojTHybBNw': {'/biz/55-south-san-jose': 5,
  '/biz/99-chicken-santa-clara': 4,
  '/biz/a-1-performance-auto-repair-sunnyvale': 5,
  '/biz/adamsons-french-dip-sunnyvale': 3,
  '/biz/ai-noodle-cupertino': 3,
  '/biz/alexanders-steakhouse-san-francisco-7': 4,
  '/biz/anchor-bay-thai-kitchen-gualala': 4,
  '/biz/anikis-sushi-fremont': 5,
  '/biz/aquamaids-bingo-santa-clara-2': 5,
  '/biz/arco-sunnyvale': 5,
  '/biz/arlanda-express-stockholm': 4,
  '/biz/asian-pearl-seafood-restaurant-fremont': 3,
  '/biz/back-a-yard-san-jose': 5,
  '/biz/backyard-bayou-union-city-2': 3,
  '/biz/bagel-street-cafe-sunnyvale': 4,
  '/biz/base-camp-pizza-south-lake-tahoe': 5,
  '/biz/beauty-fusion-studio-sunnyvale-4': 2,
  '/biz/best-buy-san-carlos-san-carlos': 4,
  '/biz/bi-rite-creamery-san-francisco': 4,
  '/biz/big-e-cafe-san-jose': 5,
  '/biz/bj

In [87]:
rev_vocab = [item for sl in list(all_reviews.values()) for item in sl.keys()]

In [92]:
rev_set = list(set(rev_vocab))

In [112]:
all_reviews_vector = []

for usr,rating in all_reviews.items():
    for key in rating.keys():
        usr_vector = []
        if key in rev_set:
            usr_vector.extend(all_reviews[k][key])
        else:
            usr_vector.extend([0])
        all_reviews_vector.append(usr_vector)

TypeError: 'int' object is not iterable

In [106]:
len(all_usr_vector)

48

In [110]:
for k,v in all_reviews.items():
    for key in v.keys():
        print(key)
    

/biz/tavu-beauty-studio-la-habra
/biz/cups-los-angeles-5
/biz/sunset-pet-and-supplies-los-angeles
/biz/feng-mao-lamb-kebab-los-angeles
/biz/lsxo-huntington-beach-2
/biz/aqua-s-houston-7
/biz/chakan-sul-lung-tang-los-angeles
/biz/twirl-swirl-los-angeles
/biz/kazunori-the-original-hand-roll-bar-santa-monica
/biz/the-seafood-spot-houston
/biz/blockheads-shavery-los-angeles
/biz/el-matador-state-beach-malibu
/biz/the-luxe-buffet-westminster-3
/biz/gjusta-los-angeles-3
/biz/salt-and-straw-los-angeles-3
/biz/star-night-los-angeles
/biz/yelp-elite-perk-bumsan-organic-milk-bar-los-angeles
/biz/we-have-noodles-los-angeles
/biz/starbucks-los-angeles-350
/biz/nobu-malibu-malibu-4
/biz/saigon-plaza-chinatown-plaza-and-dynasty-center-los-angeles
/biz/matcha-matcha-arcadia-2
/biz/fiber-art-los-angeles-13
/biz/tan-cha-los-angeles-3
/biz/emc-seafood-and-raw-bar-los-angeles
/biz/dolci-rosemead-2
/biz/edko-nails-world-champion-specialists-west-hollywood
/biz/eagle-glen-golf-club-corona
/biz/thanh-binh-r

/biz/the-centurion-lounge-san-francisco
/biz/cotogna-san-francisco
/biz/focaccia-market-bakery-san-francisco-2
/biz/dosa-on-fillmore-san-francisco-3
/biz/pizzeria-delfina-san-francisco-3
/biz/super-duper-burgers-san-francisco-5
/biz/leos-oyster-bar-san-francisco-2
/biz/lers-ros-san-francisco-2
/biz/mr-bings-san-francisco
/biz/ernies-tin-bar-petaluma
/biz/angel-island-state-park-tiburon
/biz/sams-anchor-cafe-tiburon
/biz/vignette-pizzeria-sebastopol
/biz/cellarmaker-brewing-company-san-francisco
/biz/fiorella-san-francisco
/biz/tamarind-hall-san-francisco
/biz/waxmans-restaurant-san-francisco-2
/biz/hong-kong-lounge-ii-san-francisco-3
/biz/street-taco-san-francisco
/biz/lemonade-san-francisco-3
/biz/54-mint-ristorante-italiano-san-francisco-2
/biz/bouchon-bakery-yountville
/biz/pachamama-coffee-sacramento-2
/biz/leopolds-san-francisco-2
/biz/de-afghanan-kabob-house-san-francisco-2
/biz/stanelys-donuts-sacramento
/biz/wallse-new-york-8
/biz/craft-kombucha-washington
/biz/golden-gate-urge

/biz/taste-of-sichuan-vancouver-vancouver-3
/biz/portland-farmers-market-portland-9
/biz/urban-edge-portland
/biz/basta-portland-2
/biz/elite-event-yelp-takes-it-easy-easy-company-portland
/biz/pho-thanh-vancouver
/biz/koi-fusion-portland
/biz/rudys-barbershop-portland
/biz/koji-osakaya-vancouver
/biz/los-mariachis-red-bluff-2
/biz/wheel-works-belmont-2
/biz/yeganeh-bakery-and-kafe-unik-san-jose-4
/biz/cijjo-restaurant-and-lounge-mountain-view
/biz/qq-noodle-milpitas
/biz/emazinglights-milpitas-2
/biz/jack-in-the-box-mountain-view-2
/biz/mervyns-lounge-and-restaurant-mountain-view
/biz/half-and-half-tea-express-pasadena
/biz/mr-ts-barber-shop-santa-clara
/biz/ramen-parlor-san-mateo
/biz/fogo-de-ch%C3%A3o-brazilian-steakhouse-san-jose-3
/biz/live-sushi-bar-san-francisco
/biz/round-table-pizza-los-gatos
/biz/sushi-on-the-run-los-gatos
/biz/edmonds-plaza-florist-san-mateo-2
/biz/99-chicken-santa-clara
/biz/the-cheese-steak-shop-san-jose
/biz/kitchen-kura-san-francisco
/biz/split-san-franc

/biz/yuki-sushi-santa-clara
/biz/nike-brand-las-vegas
/biz/the-java-hut-kings-beach
/biz/100-sweet-cafe-san-francisco
/biz/lollicup-sacramento-7
/biz/bombay-garden-santa-clara
/biz/ramen-tenma-san-jose
/biz/pita-pit-san-jose-3
/biz/splendid-foot-reflexology-san-jose
/biz/which-wich-san-jose
/biz/the-old-teahouse-davis
/biz/a-and-k-noodle-house-san-jose
/biz/charlies-cheesecake-works-san-jose
/biz/mayo-and-mustard-south-san-francisco-2
/biz/dragon-city-restaurant-san-francisco
/biz/twisted-chill-san-jose
/biz/red-corner-asia-los-angeles
/biz/brodard-restaurant-garden-grove-3
/biz/mobowl-mountain-view
/biz/bigg-shrimpn-food-truck-san-jose
/biz/tai-pan-dim-sum-honolulu
/biz/jade-china-san-jose-4
/biz/marukame-udon-honolulu
/biz/aston-at-the-waikiki-banyan-honolulu-2
/biz/bubble-puff-and-tea-honolulu
/biz/abc-seafood-restaurant-milpitas
/biz/willow-glen-creamery-san-jose-4
/biz/kiki-japanese-restaurant-san-francisco
/biz/downtown-san-jose-farmers-market-san-jose
/biz/yuki-sushi-san-jose
/b

In [97]:
all_reviews['ycztD9CWXLfDk4bPmA_tnw']['/biz/tavu-beauty-studio-la-habra']

5